# Web Scraping - Lockbit
Lockbit link: http://lockbitapt6vx57t3eeqjofwgcglmutr3a35nygvokja5uuccip4ykyd.onion/

## Using socks to connect to Tor proxy

In [ ]:
# Importing modules

import socks
import socket
import requests
from bs4 import BeautifulSoup

In [ ]:
#Configure socks to use Tor

from urllib.request import urlopen
socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)

old_socket = socket.socket # saving old socket info 
socket.socket = socks.socksocket

In [ ]:
# Perform DNS resolution through the socket

def getaddrinfo(*args):
    return [(socket.AF_INET, socket.SOCK_STREAM, 6, '', (args[0], args[1]))]

socket.getaddrinfo = getaddrinfo

In [ ]:
# issuing a GET request using python requests library

lockbit_link = "http://lockbitapt6vx57t3eeqjofwgcglmutr3a35nygvokja5uuccip4ykyd.onion/"
res = requests.get(lockbit_link)

In [ ]:
# Using beautifulsoup to get the website content into a nice format

soup = BeautifulSoup(res.content, 'html.parser')

In [ ]:
from __future__ import print_function
from re import sub
try:
    from urllib.request import urlopen  # using Python 3
except ImportError:
    from urllib2 import urlopen 

In [ ]:
results=soup.findAll("div",{'class':"post-title"})
website_list = []

for result in results:
    website_list.append(sub("&ldquo;|.&rdquo;","","".join(result.contents[0:1]).strip()))
    
print(website_list)

In [ ]:
# # Getting all victim companies
# blackmatter_links = ["http://blackmax7su6mbwtcyo3xwtpfxpm356jjqrs34y4crcytpw7mifuedyd.onion/",
#                     "http://blackmax7su6mbwtcyo3xwtpfxpm356jjqrs34y4crcytpw7mifuedyd.onion/?page=2&per-page=18"]

# header_list = []

# for link in blackmatter_links:
#     res = requests.get(link)
#     soup = BeautifulSoup(res.content, 'html.parser')
#     h4_tags = soup.find_all('h4')

#     for tag in h4_tags:
#         header_list.append(tag.text)

# print(header_list)
# print(len(header_list))

## Using selenium to automate Chrome browser

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame(columns=['LockBit Websites', 'Company Name', 'Industry', 'Headquarters'])
df['LockBit Websites'] = website_list

df.to_csv('LockBit.csv')

df

In [ ]:
# %pip install webdriver_manager

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys



In [ ]:
def set_viewport_size(driver, width, height):
    window_size = driver.execute_script("""
        return [window.outerWidth -  document.body.clientWidth + arguments[0],
          document.body.clientHeight - window.innerHeight + arguments[1]];
        """, width, height)
    driver.set_window_size(*window_size)

In [ ]:
socket.socket = old_socket 

s=Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service=s)

# set the viewport size to 800 x 600
# set_viewport_size(driver, 200, 200)

# driver.implicitly_wait(5)

# driver.maximize_window()

#handle cookies
driver.get('https://www.google.com')


## Data scraping from glassdoor job reviews 

In [ ]:
TAG_NAME = "tag name"
XPATH = "xpath"
CLASS_NAME = "class name"

company_list = []
industry_list = []
location_list = []
number = 1


# For ChromeDriver version 79.0.3945.16 or over
# options = webdriver.ChromeOptions()
# options.add_argument('--disable-blink-features=AutomationControlled') 
# options.add_argument("enable-automation");
# options.add_argument("--headless");
# options.add_argument("--window-size=1920,1080");
# options.add_argument("--no-sandbox");
# options.add_argument("--disable-extensions");
# options.add_argument("--dns-prefetch-disable");
# options.add_argument("--disable-gpu");
# options.add_argument("disable-infobars")


for header in website_list[0:50]: 
        
    try:
        driver.implicitly_wait(10)
#         set_viewport_size(driver, 200, 200)
        driver.get('https://www.google.com')

        search = driver.find_element(By.NAME, 'q')
        search.send_keys(header + ' glassdoor overview')
        search.send_keys(Keys.RETURN)
#         driver.maximize_window() # For maximizing window
#         driver.implicitly_wait(20) # gives an implicit wait for 20 second
        glassdoor_page = driver.find_element(By.TAG_NAME, 'h3')
        glassdoor_page.click()
        
        if "glassdoor" in driver.current_url:
            company = driver.find_element(By.TAG_NAME, 'h1')
            company_list.append(company.text)
            
            pagesource = driver.page_source
            soup = BeautifulSoup(pagesource, 'html.parser')
            labels = soup.find_all('label')
            for label in labels:
                if (label.text == "Industry:"):
                    industry = label.next_sibling.text
                    industry_list.append(industry)
                    break
            else:
                industry_list.append(np.nan)
                
            for label in labels:
                if (label.text == "Headquarters:"):
                    location = label.next_sibling.text
                    location_list.append(location)
                    break
            else:
                location_list.append(np.nan)
        else:
            company_list.append(np.nan)
            industry_list.append(np.nan)
            location_list.append(np.nan)
    except Exception as e:
        if driver.find_elements(By.TAG_NAME, 'iframe' ) !=0:
            #solve captcha manually as unable to bypass
            sleep(30)
            
            
            glassdoor_page = driver.find_element(By.TAG_NAME, 'h3')
            glassdoor_page.click()
        
            if "glassdoor" in driver.current_url:
                company = driver.find_element(By.TAG_NAME, 'h1')
                company_list.append(company.text)

                pagesource = driver.page_source
                soup = BeautifulSoup(pagesource, 'html.parser')
                labels = soup.find_all('label')
                for label in labels:
                    if (label.text == "Industry:"):
                        industry = label.next_sibling.text
                        industry_list.append(industry)
                        break
                else:
                    industry_list.append(np.nan)

                for label in labels:
                    if (label.text == "Headquarters:"):
                        location = label.next_sibling.text
                        location_list.append(location)
                        break
                else:
                    location_list.append(np.nan)    
        else:
            company_list.append(np.nan)
            industry_list.append(np.nan)
            location_list.append(np.nan)

    print(str(number) + ") " + header)
    number = number + 1
    

In [ ]:
for header in website_list[209:]: 
        
    try:
        driver.implicitly_wait(10)
#         set_viewport_size(driver, 200, 200)
        driver.get('https://www.google.com')

        search = driver.find_element(By.NAME, 'q')
        search.send_keys(header + ' glassdoor overview')
        search.send_keys(Keys.RETURN)
#         driver.maximize_window() # For maximizing window
#         driver.implicitly_wait(20) # gives an implicit wait for 20 second
        glassdoor_page = driver.find_element(By.TAG_NAME, 'h3')
        glassdoor_page.click()
        
        if "glassdoor" in driver.current_url:
            company = driver.find_element(By.TAG_NAME, 'h1')
            company_list.append(company.text)
            
            pagesource = driver.page_source
            soup = BeautifulSoup(pagesource, 'html.parser')
            labels = soup.find_all('label')
            for label in labels:
                if (label.text == "Industry:"):
                    industry = label.next_sibling.text
                    industry_list.append(industry)
                    break
            else:
                industry_list.append(np.nan)
                
            for label in labels:
                if (label.text == "Headquarters:"):
                    location = label.next_sibling.text
                    location_list.append(location)
                    break
            else:
                location_list.append(np.nan)
        else:
            company_list.append(np.nan)
            industry_list.append(np.nan)
            location_list.append(np.nan)
    except Exception as e:
        if driver.find_elements(By.TAG_NAME, 'iframe' ) !=0:
            #solve captcha manually as unable to bypass
            sleep(30)
            
            
            glassdoor_page = driver.find_element(By.TAG_NAME, 'h3')
            glassdoor_page.click()
        
            if "glassdoor" in driver.current_url:
                company = driver.find_element(By.TAG_NAME, 'h1')
                company_list.append(company.text)

                pagesource = driver.page_source
                soup = BeautifulSoup(pagesource, 'html.parser')
                labels = soup.find_all('label')
                for label in labels:
                    if (label.text == "Industry:"):
                        industry = label.next_sibling.text
                        industry_list.append(industry)
                        break
                else:
                    industry_list.append(np.nan)

                for label in labels:
                    if (label.text == "Headquarters:"):
                        location = label.next_sibling.text
                        location_list.append(location)
                        break
                else:
                    location_list.append(np.nan)    
        else:
            company_list.append(np.nan)
            industry_list.append(np.nan)
            location_list.append(np.nan)

    print(str(number) + ") " + header)
    number = number + 1

## Data cleaning of dataframe

In [ ]:
# df['Company Name'] = company_list
# df['Industry'] = industry_list
# df['Headquarters'] = location_list

df['Company Name'] = pd.Series(company_list)
df['Industry'] = pd.Series(industry_list)
df['Headquarters'] = pd.Series(location_list)


with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df)

In [ ]:
# creating bool series True for NaN values
bool_series = pd.isnull(df["Industry"])
print(bool_series)   

# filtering data
# displaying data only with Industry = NaN
noIndustry = df[bool_series]
pd.isnull(noIndustry["Industry"])

In [ ]:
len(noIndustry)

In [ ]:
# creating bool series True for NaN values
bool_series_hq = pd.isnull(df["Headquarters"])
print(bool_series_hq)   

# filtering data
# displaying data only with Hq = NaN
noHq = df[bool_series_hq]
pd.isnull(noHq["Headquarters"])

In [ ]:
updated_df = df.copy()
updated_df

## Data scraping from indeed job reviews 

In [ ]:
#delete row that has 0 google res -> Vacatures bij Groupe Schiever in Faulquemont (Frankrijk)
df.drop(45, inplace=True)

In [ ]:
df = df.drop(columns='index')


In [ ]:
#regex to search html template
import re

In [ ]:
socket.socket = old_socket 

s=Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service=s)

#handle cookies
driver.get('https://www.google.com')

In [ ]:
number = 1

for i in range(len(df)): 
    if pd.isnull(df["Industry"][i]):
        try:
#             driver.implicitly_wait(10)
#             set_viewport_size(driver, 200, 200)
            driver.get('https://www.google.com')

            search = driver.find_element(By.NAME, 'q')
            
            #search by company name if exists
            if not pd.isnull(df['Company Name'][i]):
                search.send_keys(df["Company Name"][i] + ' indeed')
            else:
                search.send_keys(df["LockBit Websites"][i] + ' indeed')
            search.send_keys(Keys.RETURN)
    #         driver.maximize_window() # For maximizing window
    #         driver.implicitly_wait(20) # gives an implicit wait for 20 second
            indeed_page = driver.find_element(By.TAG_NAME, 'h3')
            indeed_page.click()

            if "indeed" in driver.current_url:         
                pagesource = driver.page_source
                soup = BeautifulSoup(pagesource, 'html.parser')
                listitems = soup.find_all('li')
                for item in listitems:
                    if (re.search('^Industry', item.text)!=None):
                        industry = item.text[8:]
                        updated_df['Industry'][i] = industry
                        break

        except Exception as e:
            if driver.find_elements(By.TAG_NAME, 'iframe' ) !=0:
                #solve captcha manually as unable to bypass
                sleep(30)


                indeed_page = driver.find_element(By.TAG_NAME, 'h3')
                indeed_page.click()

                if "indeed" in driver.current_url:         
                    pagesource = driver.page_source
                    soup = BeautifulSoup(pagesource, 'html.parser')
                    listitems = soup.find_all('li')
                    for item in listitems:
                        if (label.text == "Industry:"):
                            industry = label.next_sibling.text
                            updated_df['Industry'][i] = industry
                            break
#             else:
#                 continue

    print(str(number) + ") " + df["LockBit Websites"][i])
    number = number + 1

In [ ]:
updated_df


In [ ]:
#decrease in NaN rows from 152 to 137
pd.isnull(updated_df["Industry"]).sum()


In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(updated_df)

In [ ]:
final_df = updated_df.dropna()
len(final_df)

In [ ]:
final_df = final_df.drop_duplicates()
len(final_df)

In [ ]:
final_df = final_df.reset_index(drop=True) # reset index

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(final_df)

In [ ]:
final_df.to_csv('LockBit_final2.csv')

# Obtaining info of companies scraped from Dark Web

In [16]:
import pandas as pd
import numpy as np
import time
import csv

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


from time import sleep, time
from random import uniform, randint
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.common.exceptions import NoSuchElementException

In [ ]:
comb = pd.read_csv('combined.csv')

In [ ]:
comb2 = pd.DataFrame(columns=['name', 'Company Name', 'Industry', 'Headquarters'])
comb2['name'] = comb['name']

comb2

In [ ]:
socket.socket = old_socket 

s=Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service=s)

# set the viewport size to 800 x 600
#set_viewport_size(driver, 400, 200)

# driver.implicitly_wait(5)

# driver.maximize_window()

#handle cookies
driver.get('https://www.google.com')
#driver.find_element(By.XPATH, '//*[@id="L2AGLb"]').click()


### Data scraping of individual company names from Glassdoor

glassdoor: https://www.glassdoor.sg/member/home/index.htm

In [ ]:
TAG_NAME = "tag name"
XPATH = "xpath"
CLASS_NAME = "class name"


number = 0


for i in range(len(comb2)): 
    header = comb2['name'][i]
    
    try:
        driver.implicitly_wait(10)
#         set_viewport_size(driver, 200, 200)
        driver.get('https://www.google.com')

        search = driver.find_element(By.NAME, 'q')
        search.send_keys(header + ' glassdoor overview')
        search.send_keys(Keys.RETURN)
#         driver.maximize_window() # For maximizing window
#         driver.implicitly_wait(20) # gives an implicit wait for 20 second
        glassdoor_page = driver.find_element(By.TAG_NAME, 'h3')
        glassdoor_page.click()
        
        if "glassdoor" in driver.current_url:
            company = driver.find_element(By.TAG_NAME, 'h1').text
            comb2['Company Name'][i] = company
            
            pagesource = driver.page_source
            soup = BeautifulSoup(pagesource, 'html.parser')
            labels = soup.find_all('label')
            for label in labels:
                if (label.text == "Industry:"):
                    industry = label.next_sibling.text
                    comb2['Industry'][i]=industry
                    break

                
            for label in labels:
                if (label.text == "Headquarters:"):
                    location = label.next_sibling.text
                    comb2['Headquarters'][i]=location
                    break

    except Exception as e:
        if driver.find_elements(By.TAG_NAME, 'iframe' ) !=0:
            #solve captcha manually as unable to bypass
            sleep(30)
            
            
            glassdoor_page = driver.find_element(By.TAG_NAME, 'h3') 
            glassdoor_page.click()
        
            if "glassdoor" in driver.current_url:
                company = driver.find_element(By.TAG_NAME, 'h1')
                comb2['Company Name'][i] = company

                pagesource = driver.page_source
                soup = BeautifulSoup(pagesource, 'html.parser')
                labels = soup.find_all('label')
                for label in labels:
                    if (label.text == "Industry:"):
                        industry = label.next_sibling.text
    #                     industry_list.append(industry)
                        comb2['Industry'][i]=industry
                        break


                for label in labels:
                    if (label.text == "Headquarters:"):
                        location = label.next_sibling.text
                        comb2['Headquarters'][i]=location
                        break

    print(str(number) + ") " + header)
    number = number + 1
    

In [ ]:
#delete row that has 0 google res -> e.g. draperyconceptsny.com

#comb2.drop(203, inplace=True)
#comb2.drop(217, inplace=True)
comb2 = comb2.reset_index(drop=True) 

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(comb2)

### Data cleaning of scraped data from Glassdoor

In [ ]:
comb2 = comb2.drop(columns='Unnamed: 0')


In [ ]:
for i in range(len(comb2)):
    if not (isinstance(comb2['Company Name'][i], str)) or comb2['Company Name'][i]=='You deserve a job that loves you back':
        comb2['Company Name'][i] = np.nan

In [ ]:
# creating bool series True for NaN values 
bool_series = pd.isnull(comb2["Headquarters"])
print(bool_series)   

# filtering data
# displaying data only with Headqurters = NaN
noHq = comb2[bool_series]
pd.isnull(noHq["Headquarters"])

In [ ]:
# creating bool series True for NaN values
bool_series = pd.isnull(comb2["Industry"])
print(bool_series)   

# filtering data
# displaying data only with Industry = NaN
noIndustry = comb2[bool_series]
pd.isnull(noIndustry["Industry"])

In [ ]:
comb2.to_csv('combined2.csv')

### Data scraping of individual company names from Indeed

Indeed: https://www.indeed.jobs/

In [ ]:
comb3 = comb2.copy()
comb3 = comb3.drop(columns='Unnamed: 0')
comb3

In [ ]:
#regex to search html template
import re

In [ ]:
#filling in industry = nan

number = 388

for i in range(388,len(comb3)): 
    if pd.isnull(comb3["Industry"][i]):
        try:
#             driver.implicitly_wait(10)
#             set_viewport_size(driver, 200, 200)
            driver.get('https://www.google.com')

            search = driver.find_element(By.NAME, 'q')
            
            #search by company name if exists
            if not pd.isnull(comb3['Company Name'][i]):
                search.send_keys(comb3["Company Name"][i] + ' indeed')
            else:
                search.send_keys(comb3["name"][i] + ' indeed')
            search.send_keys(Keys.RETURN)
    #         driver.maximize_window() # For maximizing window
    #         driver.implicitly_wait(20) # gives an implicit wait for 20 second
            indeed_page = driver.find_element(By.TAG_NAME, 'h3')
            indeed_page.click()

            if "indeed" in driver.current_url:         
                pagesource = driver.page_source
                soup = BeautifulSoup(pagesource, 'html.parser')
                listitems = soup.find_all('li')
                for item in listitems:
                    if (re.search('^Industry', item.text)!=None):
                        industry = item.text[8:]
                        comb3['Industry'][i] = industry
                        print(industry)
                        break

        except Exception as e:
            if driver.find_elements(By.TAG_NAME, 'iframe' ) !=0:
                #solve captcha manually as unable to bypass
                sleep(30)


                indeed_page = driver.find_element(By.TAG_NAME, 'h3')
                indeed_page.click()

                if "indeed" in driver.current_url:         
                    pagesource = driver.page_source
                    soup = BeautifulSoup(pagesource, 'html.parser')
                    listitems = soup.find_all('li')
                    for item in listitems:
                        if (re.search('^Industry', item.text)!=None):
                            industry = item.text[8:]
                            comb3['Industry'][i] = industry
                            print(industry)
                            break
            continue

#             else:
#                 continue

    print(str(number) + ") " + comb3["name"][i])
    number = number + 1

In [ ]:
#industry = NaN decreased from 314 to 280
pd.isnull(comb3["Industry"]).sum()

In [ ]:
#filling in headquarters = nan

number = 0

for i in range(283,284): 
    if pd.isnull(comb3["Headquarters"][i]):
        found = False
        try:
#             driver.implicitly_wait(10)
#             set_viewport_size(driver, 200, 200)
            driver.get('https://www.google.com')

            search = driver.find_element(By.NAME, 'q')
            
            #search by company name if exists
            if not pd.isnull(comb3['Company Name'][i]):
                search.send_keys(comb3["Company Name"][i] + ' indeed')
            else:
                search.send_keys(comb3["name"][i] + ' indeed')
            search.send_keys(Keys.RETURN)
    #         driver.maximize_window() # For maximizing window
    #         driver.implicitly_wait(20) # gives an implicit wait for 20 second
            indeed_page = driver.find_element(By.TAG_NAME, 'h3')
            indeed_page.click()

            if "indeed" in driver.current_url:         
                pagesource = driver.page_source
                soup = BeautifulSoup(pagesource, 'html.parser')
                listitems = soup.find_all('li')
                for item in listitems:
                    if (re.search('^Headquarters', item.text)!=None):
                        hq = item.text[12:]
                        comb3['Headquarters'][i] = hq
                        found = True
                        print(hq)
                        break
                if not found:
                    print("entered")
                    if (driver.find_elements(By.CSS_SELECTOR, '#cmp-Select-Location-label + div [data-testid="selected-value"]')!=[]):
                        print("done")
                        print(driver.find_element(By.CSS_SELECTOR, '#cmp-Select-Location-label + div [data-testid="selected-value"]')[0].text)
                        hq = driver.find_element(By.CSS_SELECTOR, '#cmp-Select-Location-label + div [data-testid="selected-value"]')[0].text
                        comb3['Headquarters'][i] = hq

        except Exception as e:
            if driver.find_elements(By.TAG_NAME, 'iframe' ) !=0:
                #solve captcha manually as unable to bypass
                sleep(30)


                indeed_page = driver.find_element(By.TAG_NAME, 'h3')
                indeed_page.click()

                if "indeed" in driver.current_url:         
                    pagesource = driver.page_source
                    soup = BeautifulSoup(pagesource, 'html.parser')
                    listitems = soup.find_all('li')
                    for item in listitems:
                        if (re.search('^Headquarters', item.text)!=None):
                            hq = item.text[12:]
                            comb3['Headquarters'][i] = hq
                            found = True
                            print(hq)
                            break
                    if not found:
                        driver.get(driver.current_url)
                        if (driver.find_elements(By.CSS_SELECTOR, '#cmp-Select-Location-label + div [data-testid="selected-value"]')!=[]):
                            hq = driver.find_element(By.CSS_SELECTOR, '#cmp-Select-Location-label + div [data-testid="selected-value"]')[0].text
                            print(hq)
                            comb3['Headquarters'][i] = hq
            continue

    print(str(number) + ") " + comb3["name"][i])
    number = number + 1

In [ ]:
#geography/headquarters = NaN decreased from 181 to 147
pd.isnull(comb3["Headquarters"]).sum()

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(comb3)

In [18]:
comb3.to_csv('combined3.csv')

NameError: name 'comb3' is not defined

In [19]:
comb4 = pd.read_csv('combined3.csv')

In [25]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(comb4)

,name,Company Name,Industry,Headquarters
0,BHARBERT,NaN,Construction,"Birmingham, AL"
1,Sigmund Software,Sigmund Software Reviews,NaN,NaN
2,Exela Technologies,Exela Technologies Reviews,NaN,United States
3,MILLS GROUP,Mills (Singapore),NaN,"Singapore, Singapore"
4,EIITNET,I.T,NaN,"Causeway Bay, Hong Kong"
5,AdaptIT,Adaptit SA,NaN,"Athens, Greece"
6,CARTEGRAPH,Cartegraph,Enterprise Software & Network Solutions,"Dubuque, IA"
7,WWSTEELE,W&W AFCO Steel,Machinery Manufacturing,"Oklahoma City, OK"
8,RALLYE-DOM,Rally,Internet & Web Services,"Portland, OR"
9,SANDO,Sando,NaN,"Málaga, Spain"


### Data cleaning of scraped data from Glassdoor

In [24]:
comb4 = comb4.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

In [ ]:
comb4['Headquarters'] = comb4['Headquarters'].replace('All', np.nan)

### Appending country code and country of each company 

In [ ]:
# import sys
# !{sys.executable} -m pip install pycountry
import pycountry

In [28]:
comb4['Country Code'] = np.nan
comb4['Country'] = np.nan

In [42]:
for co in list(pycountry.countries):
    print(co)

Birmingham, AL
Country(alpha_2='AW', alpha_3='ABW', flag='🇦🇼', name='Aruba', numeric='533')
Country(alpha_2='AF', alpha_3='AFG', flag='🇦🇫', name='Afghanistan', numeric='004', official_name='Islamic Republic of Afghanistan')
Country(alpha_2='AO', alpha_3='AGO', flag='🇦🇴', name='Angola', numeric='024', official_name='Republic of Angola')
Country(alpha_2='AI', alpha_3='AIA', flag='🇦🇮', name='Anguilla', numeric='660')
Country(alpha_2='AX', alpha_3='ALA', flag='🇦🇽', name='Åland Islands', numeric='248')
Country(alpha_2='AL', alpha_3='ALB', flag='🇦🇱', name='Albania', numeric='008', official_name='Republic of Albania')
Country(alpha_2='AD', alpha_3='AND', flag='🇦🇩', name='Andorra', numeric='020', official_name='Principality of Andorra')
Country(alpha_2='AE', alpha_3='ARE', flag='🇦🇪', name='United Arab Emirates', numeric='784')
Country(alpha_2='AR', alpha_3='ARG', flag='🇦🇷', name='Argentina', numeric='032', official_name='Argentine Republic')
Country(alpha_2='AM', alpha_3='ARM', flag='🇦🇲', name

In [44]:
for ind in range(len(comb4['Headquarters'])):
    hq = comb4['Headquarters'][ind]
    print(hq)
    
    for co in list(pycountry.countries):
        if (not pd.isnull(hq)) and (co.name in hq):
            comb4['Country Code'][ind] = co.alpha_3
            comb4['Country'][ind] = co.name


Birmingham, AL
nan
United States
Singapore, Singapore
Causeway Bay, Hong Kong
Athens, Greece
Dubuque, IA
Oklahoma City, OK
Portland, OR
Málaga, Spain
Singapore
São Paulo, Brazil
Caerphilly, United Kingdom
nan
London, United Kingdom
Kingston, Jamaica
Oakland, CA
nan
nan
United States
Singapore, Singapore
nan
Trowbridge, United Kingdom
Bogotá, Colombia
Singapore, Singapore
Canada
Moraine, OH
United States
Sydney, Australia
United States
Istanbul, Turkey
United States
Canada
United States
United States
United Kingdom
Altamonte Springs, FL
Amsterdam, Netherlands
Baton Rouge, LA
Conshohocken, PA
Canada
nan
Dublin, Ireland
San Diego, CA
nan
nan
Nieuw-Vennep, Netherlands
United States
Cincinnati, OH
Kansas City, MO
Buenos Aires, Argentina
Indianapolis, IN
Jakarta, Indonesia
Singapore
India
Osawatomie, KS
Changchun, China
Athens, Greece
Pune, India
Leinfelden-Echterdingen, Germany
Karlstein an der Thaya, Austria
Jakarta, Indonesia
Santo Domingo, Dominican Republic
Rochester, NY
Asuncion, Parag

<ipython-input-44-62794a1aa7f5>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comb4['Country Code'][ind] = co.alpha_3
<ipython-input-44-62794a1aa7f5>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comb4['Country'][ind] = co.name
/Users/magdelineng/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [46]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(comb4)

,name,Company Name,Industry,Headquarters,Country Code,Country
0,BHARBERT,NaN,Construction,"Birmingham, AL",NaN,NaN
1,Sigmund Software,Sigmund Software Reviews,NaN,NaN,NaN,NaN
2,Exela Technologies,Exela Technologies Reviews,NaN,United States,USA,United States
3,MILLS GROUP,Mills (Singapore),NaN,"Singapore, Singapore",SGP,Singapore
4,EIITNET,I.T,NaN,"Causeway Bay, Hong Kong",HKG,Hong Kong
5,AdaptIT,Adaptit SA,NaN,"Athens, Greece",GRC,Greece
6,CARTEGRAPH,Cartegraph,Enterprise Software & Network Solutions,"Dubuque, IA",NaN,NaN
7,WWSTEELE,W&W AFCO Steel,Machinery Manufacturing,"Oklahoma City, OK",NaN,NaN
8,RALLYE-DOM,Rally,Internet & Web Services,"Portland, OR",NaN,NaN
9,SANDO,Sando,NaN,"Málaga, Spain",ESP,Spain


In [50]:
comb4.to_csv('combined4.csv')

In [51]:
comb4.count()

name            488
Company Name    447
Industry        208
Headquarters    335
Country Code    189
Country         189
dtype: int64